# GWAS Tutorial: Data Exploration, QC & Association Analysis

This notebook walks through a complete genome-wide association study (GWAS) using [Hail](https://hail.is/) and a subset of the 1000 Genomes dataset — from raw VCF to association results.

**Sections:**
1. **Setup** — install Hail, download data
2. **Data Exploration** — understand the dataset structure
3. **Quality Control** — filter low-quality samples and variants
4. **Association Analysis** — run GWAS and interpret results

> Run cells from top to bottom. The setup cell installs Hail and downloads ~20 MB of data — takes ~2 minutes on first run.

## 1. Setup

In [ ]:
# Install Java (Hail's Spark backend) and Hail
!pip install hail -q

import hail as hl
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from pprint import pprint

hl.init()
from hail.plot import show
hl.plot.output_notebook()

# Download 1000 Genomes data (~20 MB)
import os
os.makedirs('data', exist_ok=True)
hl.utils.get_1kg('data/')

# Import VCF once, write to Hail's fast binary MatrixTable format
hl.import_vcf('data/1kg.vcf.bgz').write('data/1kg.mt', overwrite=True)
mt = hl.read_matrix_table('data/1kg.mt')

# Attach sample phenotypes (population, sex, caffeine consumption, ...)
table = hl.import_table('data/1kg_annotations.txt', impute=True).key_by('Sample')
mt = mt.annotate_cols(pheno=table[mt.s])
print('Loaded: %d variants x %d samples' % mt.count())

## 2. Data Exploration

### The MatrixTable

Hail stores genomic data in a **MatrixTable** — a 2D structure where:

```
              Sample1  Sample2  Sample3  ...
 chr1:100 A/T   0/1      0/0      1/1
 chr1:200 G/C   0/0      0/1      0/1
    ...          ...      ...      ...
```

- **Rows** = variants (locus + alleles)
- **Columns** = samples (sample ID `s`)
- **Entries** = genotype calls (GT, DP, GQ, AD, PL)
- **Row fields** = variant INFO annotations
- **Column fields** = sample metadata

In [ ]:
mt.rows().select().show(5)  # row key: locus + alleles

In [ ]:
mt.entry.take(3)  # entry fields: GT, DP, GQ, AD, PL

### Sample Annotations

The annotations file adds per-sample:
- `SuperPopulation` — AFR, AMR, EAS, EUR, SAS
- `Population` — 26 specific populations
- `isFemale` — biological sex
- `CaffeineConsumption` — simulated continuous phenotype (our GWAS trait)
- `PurpleHair` — simulated binary phenotype

In [ ]:
table.show(5, width=120)

In [ ]:
mt.col.describe()  # confirm pheno fields are attached

### Phenotype & Population Overview

In [ ]:
pop_counts = mt.aggregate_cols(hl.agg.counter(mt.pheno.SuperPopulation))
pprint(pop_counts)

In [ ]:
pops = sorted(pop_counts.keys())
counts = [pop_counts[p] for p in pops]
colors = ['#4e9af1', '#f4a261', '#2a9d8f', '#e76f51', '#8ecae6']

fig, ax = plt.subplots(figsize=(8, 4))
bars = ax.bar(pops, counts, color=colors, edgecolor='white')
for bar, c in zip(bars, counts):
    ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 1,
            str(c), ha='center', va='bottom', fontsize=11)
ax.set(xlabel='Super-Population', ylabel='Samples', title='Samples by Super-Population')
ax.set_ylim(0, max(counts) * 1.15)
plt.tight_layout()
plt.show()

In [ ]:
pprint(mt.aggregate_cols(hl.agg.stats(mt.pheno.CaffeineConsumption)))

In [ ]:
p = hl.plot.histogram(mt.pheno.CaffeineConsumption, bins=30,
                      title='Caffeine Consumption Distribution', legend='mg/day')
show(p)

### Variant Exploration

In [ ]:
snp_counts = mt.aggregate_rows(
    hl.agg.counter(hl.Struct(ref=mt.alleles[0], alt=mt.alleles[1]))
)
print('Top 10 allele changes:')
for k, v in Counter(snp_counts).most_common(10):
    print('  %s -> %s: %d' % (k.ref, k.alt, v))

#### Allele Frequency Spectrum

Most variants are rare — an L-shaped AFS is the hallmark of healthy genetic data under neutral evolution.

In [ ]:
afs = mt.aggregate_rows(hl.agg.hist(mt.info.AF[0], 0, 1.0, 50))
edges, freq = afs.bin_edges, afs.bin_freq
mids = [(a + b) / 2 for a, b in zip(edges, edges[1:])]
width = (edges[1] - edges[0]) * 0.9

fig, ax = plt.subplots(figsize=(9, 4))
ax.bar(mids, freq, width=width, color='steelblue', edgecolor='white', linewidth=0.5)
ax.set(xlabel='Allele Frequency (AF)', ylabel='Variants', title='Allele Frequency Spectrum')
ax.yaxis.set_major_formatter(lambda x, _: f'{int(x):,}')
plt.tight_layout()
plt.show()

#### Variant Density by Chromosome

In [ ]:
chrom_counts = mt.aggregate_rows(hl.agg.counter(mt.locus.contig))

def chrom_sort_key(c):
    n = c.replace('chr', '')
    return (0, int(n)) if n.isdigit() else (1, n)

chroms = sorted(chrom_counts.keys(), key=chrom_sort_key)
cnts = [chrom_counts[c] for c in chroms]

fig, ax = plt.subplots(figsize=(13, 4))
ax.bar(range(len(chroms)), cnts, color='teal', edgecolor='white', linewidth=0.5)
ax.set_xticks(range(len(chroms)))
ax.set_xticklabels(chroms, rotation=45, ha='right', fontsize=9)
ax.set(xlabel='Chromosome', ylabel='Variants', title='Variant Density by Chromosome')
plt.tight_layout()
plt.show()

#### Genotype Distribution

Because most variants are rare, the vast majority of entries (variant × sample pairs) should be homozygous reference (0/0).

In [ ]:
gt_counts = mt.aggregate_entries(hl.agg.counter(mt.GT.n_alt_alleles()))

labels = ['Hom Ref (0/0)', 'Het (0/1)', 'Hom Alt (1/1)']
values = [gt_counts.get(k, 0) for k in range(3)]
total = sum(values)
colors = ['#4e9af1', '#f4a261', '#e76f51']

fig, (ax, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ax.bar(labels, values, color=colors, edgecolor='white')
ax.set(ylabel='Count', title='Genotype Counts')
ax.yaxis.set_major_formatter(lambda x, _: f'{int(x):,}')
ax2.pie(values, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
ax2.set_title('Genotype Proportions')
plt.suptitle('Genotype Distribution Across All Entries', fontsize=13)
plt.tight_layout()
plt.show()
for label, v in zip(labels, values):
    print('%s: %d (%.1f%%)' % (label, v, 100 * v / total))

---
**🔍 Exercise 1:** The bar chart above shows counts for the 5 super-populations. Modify it to show the breakdown by the more specific `Population` field. How many distinct populations are in this dataset?

> *Hint: change `mt.pheno.SuperPopulation` → `mt.pheno.Population` in the `agg.counter` call and re-run the bar chart.*

In [ ]:
# Exercise 1 — your code here


## 3. Quality Control

Before running a GWAS, we remove low-quality samples, unreliable genotype calls, and problematic variants. Poor-quality data leads to spurious associations and inflated p-values.

### Sample QC

`hl.sample_qc()` computes per-sample metrics:

| Metric | Meaning |
|---|---|
| `call_rate` | Fraction of non-missing genotype calls |
| `dp_stats.mean` | Mean read depth |
| `gq_stats.mean` | Mean genotype quality |

Samples with low call rate or depth are removed.

In [ ]:
mt = hl.sample_qc(mt)

In [ ]:
p = hl.plot.histogram(mt.sample_qc.dp_stats.mean, range=(0, 30), bins=30,
                      title='Mean Read Depth per Sample', legend='Mean DP')
show(p)

In [ ]:
p = hl.plot.histogram(mt.sample_qc.call_rate, range=(0.88, 1.0), bins=30,
                      title='Call Rate per Sample', legend='Call Rate')
show(p)

In [ ]:
p = hl.plot.histogram(mt.sample_qc.gq_stats.mean, range=(10, 70), bins=30,
                      title='Mean GQ per Sample', legend='Mean GQ')
show(p)

In [ ]:
p = hl.plot.scatter(mt.sample_qc.dp_stats.mean, mt.sample_qc.call_rate,
                    xlabel='Mean DP', ylabel='Call Rate',
                    title='Mean Depth vs. Call Rate')
show(p)

In [ ]:
n_before = mt.count_cols()
mt = mt.filter_cols((mt.sample_qc.dp_stats.mean >= 4) & (mt.sample_qc.call_rate >= 0.97))
print('Samples: %d → %d (removed %d)' % (n_before, mt.count_cols(), n_before - mt.count_cols()))

### Allelic Balance Filter

Individual genotype calls can be unreliable even in passing samples. The **allelic balance (AB)** — fraction of reads supporting the alt allele — should be:
- ~0 for hom ref (0/0)
- ~0.5 for het (0/1)
- ~1 for hom alt (1/1)

In [ ]:
ab = mt.AD[1] / hl.sum(mt.AD)
filter_ab = (
    (mt.GT.is_hom_ref() & (ab <= 0.1)) |
    (mt.GT.is_het()     & (ab >= 0.25) & (ab <= 0.75)) |
    (mt.GT.is_hom_var() & (ab >= 0.9))
)
fraction_filtered = mt.aggregate_entries(hl.agg.fraction(~filter_ab))
print('Setting %.2f%% of entries to missing (failed AB filter).' % (fraction_filtered * 100))
mt = mt.filter_entries(filter_ab)

---
**🔍 Exercise 2:** The current het allelic balance window is **0.25–0.75**. What fraction of entries would *additionally* be removed if you tightened it to **0.30–0.70**?

> *Hint: compute `hl.agg.fraction(~filter_ab_strict)` using the stricter condition on `mt` before calling `filter_entries`. The fraction that fails the strict but not the current filter is the difference.*

In [ ]:
# Exercise 2 — your code here


### Variant QC

`hl.variant_qc()` computes per-variant metrics:

| Metric | Meaning |
|---|---|
| `AF[1]` | Alternate allele frequency in this dataset |
| `call_rate` | Fraction of samples with a call |
| `p_value_hwe` | Hardy-Weinberg equilibrium test p-value |

In [ ]:
mt = hl.variant_qc(mt)

In [ ]:
af_hist = mt.aggregate_rows(hl.agg.hist(mt.variant_qc.AF[1], 0, 0.5, 50))
edges, freq = af_hist.bin_edges, af_hist.bin_freq
mids = [(a + b) / 2 for a, b in zip(edges, edges[1:])]
width = (edges[1] - edges[0]) * 0.9

fig, ax = plt.subplots(figsize=(9, 4))
ax.bar(mids, freq, width=width, color='steelblue', edgecolor='white', linewidth=0.5)
ax.axvline(x=0.01, color='red', linestyle='--', linewidth=1.5, label='AF = 1% filter')
ax.set(xlabel='Alternate Allele Frequency', ylabel='Variants',
       title='MAF Distribution (before filtering)')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
hwe_hist = mt.aggregate_rows(
    hl.agg.filter(hl.is_defined(mt.variant_qc.p_value_hwe),
                  hl.agg.hist(-hl.log10(mt.variant_qc.p_value_hwe), 0, 10, 40))
)
edges, freq = hwe_hist.bin_edges, hwe_hist.bin_freq
mids = [(a + b) / 2 for a, b in zip(edges, edges[1:])]
width = (edges[1] - edges[0]) * 0.9

fig, ax = plt.subplots(figsize=(9, 4))
ax.bar(mids, freq, width=width, color='darkorange', edgecolor='white', linewidth=0.5)
ax.axvline(x=6, color='red', linestyle='--', linewidth=1.5, label='p = 1e-6 filter')
ax.set(xlabel='-log10(HWE p-value)', ylabel='Variants',
       title='HWE P-value Distribution (before filtering)')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
n_vars_before = mt.count_rows()
mt = mt.filter_rows((mt.variant_qc.AF[1] > 0.01) & (mt.variant_qc.p_value_hwe > 1e-6))
n_vars_after = mt.count_rows()
print('Variants: %d → %d (removed %d, %.1f%%)' % (
    n_vars_before, n_vars_after,
    n_vars_before - n_vars_after,
    100 * (n_vars_before - n_vars_after) / n_vars_before
))

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
bars = ax.bar(['Before QC', 'After QC'], [n_vars_before, n_vars_after],
              color=['#d9534f', '#5cb85c'], edgecolor='white', width=0.5)
for bar, v in zip(bars, [n_vars_before, n_vars_after]):
    ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 20,
            f'{v:,}', ha='center', va='bottom', fontsize=12, fontweight='bold')
ax.set(ylabel='Variants', title='Variants Before and After QC')
ax.set_ylim(0, n_vars_before * 1.15)
plt.tight_layout()
plt.show()

In [ ]:
n_variants, n_samples = mt.count()
print('After QC: %d variants x %d samples' % (n_variants, n_samples))

## 4. Association Analysis

### Population Stratification — PCA

The 1000 Genomes dataset spans 5 continental populations (AFR, AMR, EAS, EUR, SAS). If we ignore this, variants that differ in frequency across populations will appear associated with caffeine — not because they affect it, but because ancestry correlates with both genotype and phenotype.

We compute **Principal Components (PCs)** of the genotype matrix to capture ancestry, then include them as covariates in regression.

In [ ]:
eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt.GT)
pprint(eigenvalues[:5])  # variance explained by each PC

In [ ]:
mt = mt.annotate_cols(scores=pcs[mt.s].scores)

In [ ]:
p = hl.plot.scatter(mt.scores[0], mt.scores[1],
                    label=mt.pheno.SuperPopulation,
                    title='PCA — PC1 vs PC2', xlabel='PC1', ylabel='PC2')
show(p)

In [ ]:
p = hl.plot.scatter(mt.scores[1], mt.scores[2],
                    label=mt.pheno.SuperPopulation,
                    title='PCA — PC2 vs PC3', xlabel='PC2', ylabel='PC3')
show(p)

### Naive GWAS (no covariates)

As a baseline, we run linear regression without any population covariates — this will show inflation because ancestry is not accounted for.

In [ ]:
gwas_naive = hl.linear_regression_rows(
    y=mt.pheno.CaffeineConsumption,
    x=mt.GT.n_alt_alleles(),
    covariates=[1.0]  # intercept only
)

In [ ]:
p = hl.plot.qq(gwas_naive.p_value, title='QQ Plot — naive (no covariates)')
show(p)

In [ ]:
p = hl.plot.manhattan(gwas_naive.p_value, title='Manhattan Plot — naive (no covariates)')
show(p)

**Genomic inflation factor (λ)** is shown in the QQ plot. It measures how inflated the test statistics are:
- λ ≈ 1.0 → well-calibrated
- λ > 1.05 → inflation (likely population stratification)
- λ < 1.0 → deflation (over-correction)

The naive GWAS should show λ >> 1.

### Corrected GWAS (sex + PC1–3 as covariates)

Including PCs as covariates removes the ancestry signal and brings λ close to 1.

In [ ]:
gwas = hl.linear_regression_rows(
    y=mt.pheno.CaffeineConsumption,
    x=mt.GT.n_alt_alleles(),
    covariates=[1.0, mt.pheno.isFemale, mt.scores[0], mt.scores[1], mt.scores[2]]
)
print('Tested: %d variants' % gwas.count())

In [ ]:
p = hl.plot.qq(gwas.p_value, title='QQ Plot — corrected (sex + PC1-3)')
show(p)

In [ ]:
p = hl.plot.manhattan(gwas.p_value, title='Manhattan Plot — corrected (sex + PC1-3)')
show(p)

### Top Hits

Since `CaffeineConsumption` is a simulated phenotype, we don't expect real hits — but this is the workflow you'd use on real data.

In [ ]:
gwas.order_by(gwas.p_value).select('p_value', 'beta', 'standard_error', 't_stat').show(20, width=120)

In [ ]:
top = gwas.order_by(gwas.p_value).take(20)
betas = np.array([r.beta for r in top])
ses   = np.array([r.standard_error for r in top])
labels = ['%s/%s' % (r.locus, r.alleles[1]) for r in top]

fig, ax = plt.subplots(figsize=(8, 7))
y = np.arange(len(top))
ax.errorbar(betas, y, xerr=1.96 * ses, fmt='o', color='steelblue',
            capsize=3, linewidth=1, markersize=5)
ax.axvline(0, color='grey', linestyle='--', linewidth=1)
ax.set_yticks(y)
ax.set_yticklabels(labels, fontsize=8)
ax.set(xlabel='Effect size (beta) ± 95% CI', title='Top 20 Variants by P-value')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

---
**🔍 Exercise 3:** We used `hl.linear_regression_rows` for the continuous `CaffeineConsumption` trait. The dataset also has a binary trait: `PurpleHair`. Run a **logistic GWAS** using `hl.logistic_regression_rows` and plot the Manhattan. Do the top hits overlap with caffeine?

> *Starter code:*
> ```python
> gwas_binary = hl.logistic_regression_rows(
>     test='wald',
>     y=mt.pheno.PurpleHair,
>     x=mt.GT.n_alt_alleles(),
>     covariates=[1.0, mt.pheno.isFemale, mt.scores[0], mt.scores[1], mt.scores[2]]
> )
> ```

In [ ]:
# Exercise 3 — your code here


## Summary

In this tutorial you:

- ✅ Downloaded and imported the 1000 Genomes VCF into a Hail MatrixTable
- ✅ Explored the dataset: MatrixTable structure, population composition, allele frequency spectrum, genotype distribution
- ✅ Applied sample QC (depth, call rate), allelic balance filtering, and variant QC (MAF, HWE)
- ✅ Ran PCA to capture population structure
- ✅ Compared a naive vs. covariate-corrected GWAS, and observed the effect of stratification on λ
- ✅ Identified and visualised the top associated variants